# Custom Layers
Source: https://www.tensorflow.org/alpha/tutorials/eager/custom_layers

In [1]:
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

## Layers: common sets of useful operations

In [2]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in the documentation. It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.


In [3]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using 
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.2590437 ,  0.21691412, -0.45973426, -0.533728  , -0.4305027 ,
          0.09812999, -0.27044576, -0.257358  ,  0.0051018 ,  0.39474005],
        [ 0.1851781 ,  0.11184257,  0.03325564, -0.58809835, -0.35753268,
          0.3647654 , -0.5471086 ,  0.35068005,  0.4582886 ,  0.35093552],
        [ 0.08469677,  0.1407538 , -0.60918665,  0.48853916, -0.569168  ,
          0.5360412 ,  0.6251305 ,  0.33964157,  0.5688638 , -0.30087864],
        [-0.11585826,  0.5057865 , -0.13786426,  0.17978483, -0.56006783,
         -0.50721097,  0.4939235 ,  0.41439885, -0.36684904,  0.12543094],
        [-0.04391342, -0.52856046, -0.5648074 ,  0.19395542,  0.10447741,
          0.12701923, -0.5590389 ,  0.01395297, -0.12737793,  0.39886147]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [5]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.2590437 ,  0.21691412, -0.45973426, -0.533728  , -0.4305027 ,
          0.09812999, -0.27044576, -0.257358  ,  0.0051018 ,  0.39474005],
        [ 0.1851781 ,  0.11184257,  0.03325564, -0.58809835, -0.35753268,
          0.3647654 , -0.5471086 ,  0.35068005,  0.4582886 ,  0.35093552],
        [ 0.08469677,  0.1407538 , -0.60918665,  0.48853916, -0.569168  ,
          0.5360412 ,  0.6251305 ,  0.33964157,  0.5688638 , -0.30087864],
        [-0.11585826,  0.5057865 , -0.13786426,  0.17978483, -0.56006783,
         -0.50721097,  0.4939235 ,  0.41439885, -0.36684904,  0.12543094],
        [-0.04391342, -0.52856046, -0.5648074 ,  0.19395542,  0.10447741,
          0.12701923, -0.5590389 ,  0.01395297, -0.12737793,  0.39886147]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers

The best way to implement your own layer is extending the tf.keras.Layer class and implementing: 

- ` __init__` , where you can do all input-independent initialization 
- `build`, where you know the shapes of the input tensors and can do the rest of the initialization 
- `call`, where you do the forward computation



In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[int(input_shape[-1]), 
                                           self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.03631818, -0.58750993,  0.10124737, -0.11126506,  0.52455   ,
         0.03279692,  0.2219826 ,  0.4701069 , -0.33879772, -0.01316237],
       [-0.5861447 ,  0.01428092, -0.21908426, -0.6169046 ,  0.26401162,
         0.18212461,  0.51165444,  0.41104776, -0.333844  ,  0.09821153],
       [ 0.02781546,  0.22092885,  0.59590393, -0.45599303,  0.18668795,
         0.4264421 ,  0.3472113 ,  0.5737639 , -0.370194  ,  0.13730425],
       [ 0.01770127,  0.5604488 ,  0.33765596,  0.3793909 , -0.10074466,
        -0.2970694 ,  0.21532738,  0

Note that you don't have to wait until build is called to create your variables, you can also create them in __init__.



## Models: composing layers

In [7]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization_v2/gamma:0', 'resnet_identity_block/batch_normalization_v2/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_v2_1/gamma:0', 'resnet_identity_block/batch_normalization_v2_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_v2_2/gamma:0', 'resnet_identity_block/batch_normalization_v2_2/beta:0']


Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`



In [8]:
 my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), 
                                                      input_shape=(
                                                          None, None, 3)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(2, 1,
                                                      padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(3, (1, 1)),
                               tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=726, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [ ]:
s